id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

Loading the dataset

In [2]:
import pickle
dataset = pickle.load(open("datasets/n 300 - seed 1234 - time 2019-06-19-153241.pickle","rb"))

Describing the data

Running the model

In [6]:
from src.main import Mixedlogit, datasetloader 

dataset = datasetloader("n 1000 - seed 1234 - time 2019-06-20-134806.pickle")

mixedlogit = Mixedlogit(dataset, "bigbatch")

# #ndraws 50
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=1, save="Base/parallel-1", start=0, end=200, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=1, save="Base/parallel-2", start=200, end=400, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=1, save="Base/parallel-3", start=400, end=600, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=1, save="Base/parallel-4", start=600, end=800, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=1, save="Base/parallel-5", start=800, end=1000, parallel=True)

# #ndraws 100
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=1, save="Base/parallel-1", start=0, end=200, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=1, save="Base/parallel-2", start=200, end=400, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=1, save="Base/parallel-3", start=400, end=600, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=1, save="Base/parallel-4", start=600, end=800, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=1, save="Base/parallel-5", start=800, end=1000, parallel=True)

In [ ]:
from src.main import Mixedlogit, datasetloader 

dataset = datasetloader("n 1000 - seed 1234 - time 2019-06-20-134806.pickle")

mixedlogit = Mixedlogit(dataset, "bigbatch")


mixedlogit.fit(drawtype="pseudo", n_draws=250, verbose=1, save="Big/250", start=0, end=200, parallel=True)

mixedlogit.fit(drawtype="pseudo", n_draws=500, verbose=1, save="Big/500", start=0, end=200, parallel=True)

mixedlogit.fit(drawtype="pseudo", n_draws=1000, verbose=1, save="Big/1000", start=0, end=200, parallel=True)

mixedlogit.fit(drawtype="pseudo", n_draws=2000, verbose=1, save="Big/2000", start=0, end=200, parallel=True)

Fitting 200 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 12.8min
